<a href="https://colab.research.google.com/github/abhisheksingh1234/DataScience/blob/master/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.6 MB/s eta 0:00:00


In [11]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from torch.utils.data import DataLoader, Dataset

# Load pre-trained DistilBERT model and tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)  # num_labels can be fine-tuned for your task
max_length = 512
# Code for chatbot dataset
class ChatbotDataset(Dataset):
    def __init__(self, data, max_length):
        self.data = data
        self.max_length = max_length  # Specify the maximum sequence length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        encoding = tokenizer(
            item['text'],
            item['response'],
            padding='max_length',  # Pad sequences to max_length
            truncation=True,
            max_length=self.max_length,  # Set the maximum sequence length
            return_tensors='pt'
        )
        input_ids = encoding['input_ids']
        attention_mask = encoding['attention_mask']
        label = torch.tensor(item['label'])
        return {'input_ids': input_ids.squeeze(), 'attention_mask': attention_mask.squeeze(), 'label': label}

# Example chatbot data
chatbot_data = [
    {'text': "I need some urgent help", 'response': "Sure, how can I help you today", 'label': 1},
    {'text': "How are you?", 'response': "I'm fine, thank you.", 'label': 1},
]

# Create DataLoader for training
chatbot_dataset = ChatbotDataset(chatbot_data, max_length)
train_loader = DataLoader(chatbot_dataset, batch_size=32, shuffle=True)

# Training loop
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

#Directly get the probability from logits
epochs = 10
for epoch in range(epochs):
    for batch in train_loader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()

# Save the trained DistilBERT model for inference
model.save_pretrained('chatbot_distilbert_model')

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
